## Contaminación del aire Madrid.

Queremos trabajar sobre estos datos para ver después como afecta la contaminación (si es que afecta) en el uso de las bicicletas. Para esto necesitamos hacer el trabajo previo de la limpieza y transformación de estos datos. Nos vamos a centrar solamente en algunas estaciones.

Los valores que importan son los siguientes:

The AQI level is based on the level of six atmospheric pollutants, namely sulfur dioxide (SO2), nitrogen dioxide (NO2), suspended particulates smaller than 10 μm in aerodynamic diameter (PM10)suspended particulates smaller than 2.5 μm in aerodynamic diameter (PM2.5)，carbon monoxide (CO), and ozone (O3) measured at the monitoring stations throughout each city

Los códigos del excel de datos son los siguientes:

* 09 - PM 2.5
* 10 - PM10
* 14 - Ozono O3
* 08 - Dioxido de nitrogeno NO2
* 01 - Dioxido de Azufre SO2
* 06 - Monoxido de carbono CO

**Códigos de estaciones:**

Más céntricas
* Retiro: 49
* Plaza del carmen: 35
* Castellana: 48

En zonas mas alejadas está:
* El pardo: 58
* Casa de campo: 24


**SOLO SON VALIDOS LOS VALORES QUE TIENEN UNA V**

Tomamos valores del 2015 al 2018

Importamos librerías que vamos a utilizar

In [ ]:
import pandas as pd
from datetime import date, datetime, timedelta
import re
import numpy as np

Importamos los datos por año que previamente descargamos de la página de datos abiertos de Madrid para el periodo 2015-2018.

In [ ]:
data2018 = pd.read_excel('../dat/calidadaire2018.xlsx')
data2017 = pd.read_excel('../dat/calidadaire2017.xlsx') 
data2016 = pd.read_excel('../dat/calidadaire2016.xlsx') 
data2015 = pd.read_excel('../dat/calidadaire2015.xlsx') 
frames = [data2018, data2017, data2016, data2015]

data = pd.concat(frames)

Los datos tienen esta estructura. Los días son las columnas pero también están relacionados con otra columna que identifica si es un valor válido o no.

In [ ]:
data.head()

Elimino las columns que ya se que no sirven.

In [ ]:
del data['PROVINCIA']
del data['MUNICIPIO']
del data['PUNTO_MUESTREO']

## Filtro estaciones y magnitudes de mi interés

Solo quiero quedarme con una parte de la información. Las magnitudes más importantes y un conjunto de estaciones que representen medidas céntricas de Madrid y de las zonas fueras del centro.

In [ ]:
estaciones_utilizar = [24,35,48,49,58] #Códigos de las estaciones descritos al inicio del documento
magnitudes_interes = [1,6,8,9,10,14] #Códigos de las magnitudes importantes, descritas al inicio del documento
data_copy = data.copy()
data_copy = data_copy[data_copy.ESTACION.isin(estaciones_utilizar)]
data_copy = data_copy[data_copy.MAGNITUD.isin(magnitudes_interes)]

### Convierto datos a formato largo

Asi me quedan los dias y los valores como filas

In [ ]:
data_formatted = pd.melt(data_copy, id_vars =['ESTACION', 'MAGNITUD', 'ANO', 'MES'])
data_formatted.head()

### Transformación de datos

Necesitaría crearme una columna con la fecha, ya que tengo mes, año y día por separado. Antes necesito limpiar el día ya que tiene una D o una V delante.

Luego voy a separar el dataframe según V o D y voy a cruzarlo por la columna fecha creada.

**Fechas con error**

Las fechas que dan error cuando se quieren castear a tipo datetime (porque no existen. ej: 30/02) las pongo como 1/1/1900 (porque ya se que no existen) para después borrarlas.

In [ ]:
def get_date_from_columns(row):
    try:
        year = row.ANO
        month = row.MES
        day = int(re.findall('[0-9]+', row.variable)[0])
        return datetime(year, month, day)
    except:
        #DA ERROR EN DIAS QUE NO EXISTEN. Los voy a borrar despues
        return datetime(1900,1,1)

def convert_to_letter(variable):
    return re.findall('[A-Z]+',variable)[0]

data_formatted['date'] = data_formatted.apply(get_date_from_columns, axis=1)
data_formatted['variable'] = data_formatted.variable.apply(convert_to_letter)

#Elimino las fechas que están mal-
data_formatted = data_formatted[data_formatted.date != date(1900,1,1)]

In [ ]:
data_formatted.head(1)

### Separo dataframe y lo cruzo por fecha, magnitud y estación

Separo el dataframe según el tipo D o V y lo cruzo para saber que días tienen valores válidos y cuales no. Elimino los que no son válidos. Filtro por aquellos con valor 'V'

In [ ]:
data_formatted_D = data_formatted[data_formatted.variable == 'D'].sort_values(['ESTACION','MES', 'MAGNITUD'])
data_formatted_V = data_formatted[data_formatted.variable == 'V'].sort_values(['ESTACION','MES', 'MAGNITUD'])

#Cruzo dataframes que separé
data_formatted = data_formatted_D.merge(data_formatted_V, on=['date','MAGNITUD', 'ESTACION'])

#Filtro por aquellos días que tengan la V y elimino los que tienen la 'N' de no válido
data_formatted = data_formatted[data_formatted.value_y == 'V']

### Transformación de columnas

Elimino columnas que no me sirven y cambio nombres para que sean mas descriptivos

In [ ]:
#Borro columnas que no me interesan.
del(data_formatted['variable_y'])
del(data_formatted['value_y'])
del(data_formatted['MES_y'])
del(data_formatted['ANO_y'])
del(data_formatted['variable_x'])

#Cambio nombre de las columnas y converto medicion a numero.
data_formatted.columns = ['estacion', 'magnitud', 'ano', 'mes', 'medicion','fecha']
data_formatted["medicion"] = pd.to_numeric(data_formatted["medicion"])

data_bd = data_formatted.copy() #Lo guardo para después

La tabla resultado se ve a continuación.

In [ ]:
data_formatted.head()

## Reemplazo magnitudes y estaciones por su nombre

Las magnitudes y las estaciones están identificadas por un número. Para saber a qué corresponden les cambio los números por su valor correspondiente

In [ ]:
def get_magnitud_name(value):
    if value == 1:
        return 'Dioxido de Azufre SO2'
    elif value == 6:
        return 'Monoxido de carbono CO'
    elif value == 8:
        return 'Dioxido de nitrogeno NO2'
    elif value == 9:
        return 'PM 2.5'
    elif value == 10:
        return 'PM10'
    elif value == 14:
        return 'Ozono O3'

def get_station_name(value):
    if value == 49:
        return 'Retiro'
    elif value == 35:
        return 'Plaza del carmen'
    elif value == 48:
        return 'Castellana'
    elif value == 58:
        return 'El Pardo'
    elif value == 24:
        return 'Casa de campo'
            
data_formatted['magnitud'] = data_formatted.magnitud.apply(get_magnitud_name)
data_formatted['estacion'] = data_formatted.estacion.apply(get_station_name)

## Visualización de datos

Importo librerias para gráficos

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 14, 10
sns.set_context('paper', font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.despine()
sns.set_style("dark")

**Valores máximos para cada magnitud por año**.

Quiero ver para hacerme la idea, cuales son los valores máximos de todas las magnitudes cada año.

In [ ]:
data_tmp = data_formatted.copy()
valores_max = data_tmp.groupby(['magnitud', 'ano']).medicion.max()
data_tmp = pd.DataFrame(valores_max).reset_index()


plt.xlabel('Año')
plt.ylabel('Medición')
plt.title('Máximo valor contaminantes por año') 
out = sns.barplot(x='ano', y ='medicion' , data=data_tmp, hue='magnitud', palette="Set3", ci=0)


En el gráfico se ve que mas o menos los máximos se mantienen en los últimos 4 años. Estaría bueno ver como ha sido la distribución por ejemplo del Dióxido de nitrógeno (en promedio por mes), que es por el cual se activan los protocolos de contaminación

In [ ]:
data_tmp = data_formatted.copy()
data_tmp = data_tmp[data_tmp.magnitud == 'Dioxido de nitrogeno NO2']

plt.title('Distribucion Dioxido de nitrogeno NO2 por año') 
out = sns.distplot(data_tmp[data_tmp.ano == 2018].medicion, hist=False, kde_kws={"color": "k", "lw": 3, "label": "2018"})
out = sns.distplot(data_tmp[data_tmp.ano == 2017].medicion, hist=False, kde_kws={"color": "y", "lw": 3, "label": "2017"})
out = sns.distplot(data_tmp[data_tmp.ano == 2016].medicion, hist=False, kde_kws={"color": "g", "lw": 3, "label": "2016"})
out = sns.distplot(data_tmp[data_tmp.ano == 2015].medicion, hist=False, kde_kws={"color": "m", "lw": 3, "label": "2015"})

Parecería que los niveles han ido mejorando y que en el 2018 los valores se encuentran mas a la izquierda del gráfico.

Quiero ver la media mes a mes entre estos años para ver si han existido mejoras.

In [ ]:
data_tmp = data_formatted.copy()
data_tmp = data_tmp[(data_tmp.magnitud == 'Dioxido de nitrogeno NO2')]
out = sns.lineplot(x='mes', y ='medicion' , data=data_tmp, hue='ano', estimator= np.mean, palette="Set2", ci=0)
plt.xlabel('Mes')
plt.ylabel('Medición')
plt.title('Media Dioxido de nitrogeno NO2 por mes 2015 - 2018 ') 
plt.show()

La media del Dióxido de nitrogeno NO2 en el 2018 es más baja en más de la mitad de los meses con respecto a los años anteriores, lo cual supone una mejora en este sentido, determinado mas que nada por la disminución en aquellos meses como noviembre, diciembre y enero donde los niveles son más altos en general.

### Activación Protocolos de contaminación 2018.

En 2018 los protocolos de contaminación se activaron 4 veces por Dióxido de Nitrogeno.

* NO2 del 23 al 24 de enero
* NO2 del 5 al 7 de diciembre de 2018
* NO2 del 11 al 12 de diciembre de 2018
* NO2 del 25 al 28 de diciembre de 2018

Los valores de esos días para retiro fueron:

In [ ]:
data_tmp = data_formatted.copy()
data_tmp = data_tmp[(data_tmp.magnitud == 'Dioxido de nitrogeno NO2') & (data_tmp.estacion == 'Retiro') & 
         ((data_tmp.fecha == datetime(2018,12,5)) | (data_tmp.fecha == datetime(2018,12,12)) 
          | (data_tmp.fecha == datetime(2018,12,25))| (data_tmp.fecha == datetime(2018,12,6))
         | (data_tmp.fecha == datetime(2018,12,26)) | (data_tmp.fecha == datetime(2018,12,27))
         | (data_tmp.fecha == datetime(2018,12,28)) | (data_tmp.fecha == datetime(2018,1,23))
         | (data_tmp.fecha == datetime(2018,1,24)) | (data_tmp.fecha == datetime(2018,12,7)))]
data_tmp

### Qué pasaba esos días en las estaciones fuera del centro?

Quiero ver si hay una diferencia significativa en los días que se activa el protocolo entre por ejemplo Retiro y El Pardo, que están relativamente cerca en cuanto a distancia pero son muy distintas en cuanto a densidad de población y tráfico.

In [ ]:
data_tmp2 = data_formatted.copy()
data_tmp2 = data_tmp2[(data_tmp2.magnitud == 'Dioxido de nitrogeno NO2') & (data_tmp2.estacion == 'El Pardo') & 
         ((data_tmp2.fecha == datetime(2018,12,5)) | (data_tmp2.fecha == datetime(2018,12,12)) 
          | (data_tmp2.fecha == datetime(2018,12,25))| (data_tmp2.fecha == datetime(2018,12,6))
         | (data_tmp2.fecha == datetime(2018,12,26)) | (data_tmp2.fecha == datetime(2018,12,27))
         | (data_tmp2.fecha == datetime(2018,12,28)) | (data_tmp2.fecha == datetime(2018,1,23))
         | (data_tmp2.fecha == datetime(2018,1,24)) | (data_tmp2.fecha == datetime(2018,12,7)))]
data_tmp2

In [ ]:
out = sns.lineplot(x='fecha', y ='medicion' , data=data_tmp2, palette="Set2", ci=3)
out = sns.lineplot(x='fecha', y ='medicion' , data=data_tmp, palette="Set1", ci=10)
plt.xlabel('Fecha')
plt.ylabel('Medición')
plt.title('Mediciones en días de protocolos de contaminación Retiro vs El Pardo') 
plt.show()

Las diferencias son bastante significativas.
Si hago el promedio anual entre ambas estaciones veo los siguientes resultados, donde el promedio de retiro es anualmente casi el doble.

In [ ]:
data_tmp = data_formatted.copy()
data_tmp = data_tmp[(data_tmp.magnitud == 'Dioxido de nitrogeno NO2') & (data_tmp.estacion.isin(['Retiro','El Pardo']))] 

plt.title('Promedio anual estaciones céntricas y fuera de madrid ') 
out = sns.barplot(x='ano', y ='medicion' , data=data_tmp, hue='estacion', estimator= np.mean, palette="Set2", ci=0)
plt.xlabel('Año')
plt.ylabel('Medición')
plt.show()

## Contaminantes promedio por estación

Para ver el comportamiento promedio de los contaminantes según la estación del año, creamos una columna que dada la fecha nos determine la estación del año.

In [ ]:
def get_season(row):
    date = row.fecha
    if date >= datetime(date.year, 3, 21) and date <= datetime(date.year, 6, 20):
        return 'Primavera'
    elif date >= datetime(date.year, 6, 21) and date <= datetime(date.year, 9 , 20):
        return 'Verano'
    elif date >= datetime(date.year, 9, 21) and date <= datetime(date.year, 12 , 20):
        return 'Otoño'
    else:
        return 'Invierno'
    
data_tmp = data_formatted.copy()    
data_tmp['estacion_del_año'] = data_tmp.apply(get_season, axis = 1)

Vemos en la siguiente gráfica que en zonas céntricas, el dioxido de nitrogeno tiene valores en promedio que son superiores en Otoño y en Invierno, lo cual tiene sentido en cuanto a los protocolos de contaminación activados.

In [ ]:
data_tmp = data_tmp[(data_tmp.estacion.isin(['Retiro','Plaza del carmen', 'Castellana']))] 
plt.title('Promedio contaminantes por estación del año en zonas céntricas') 
out = sns.barplot(x='estacion_del_año', y ='medicion' , data=data_tmp, estimator = np.mean, hue='magnitud', palette="Set1", ci=0)
plt.xlabel('Estación')
plt.ylabel('Medición')
plt.show()

## Guardo info a BD

Guardo la información en base de datos para utilizarla después en el TFM.
Solo vamos a guardar los datos de retiro.

Está comentada la conexión para no ejecutarlo de nuevo sin querer ya que no hay restricciones para uqe no vuelva a insertarse los datos por el momento. **Por eso da error si se ejecuta**

Fue necesario volver a hacer un pivot sobre los datos, pudiendo solo guardar el dioxido y el ozono que son las magnitudes que se registran para Retiro. Nos queda ver de que estación cercana podemos obtener el resto de las magnitudes.

In [ ]:
data_to_insert = data_bd.copy()
data_to_insert = data_to_insert[data_to_insert.estacion == 49]

In [ ]:
import math
def remove_Nan(element):
    if math.isnan(element):
        return 0
    else:
        return element

data_to_insert = data_to_insert.pivot_table(index=['fecha', 'ano', 'mes', 'estacion'], columns='magnitud', values='medicion').reset_index()
data_to_insert.columns = [ 'fecha', 'ano', 'mes', 'estacion','dioxido', 'ozono']
data_to_insert['dioxido'] = data_to_insert.dioxido.apply(remove_Nan)
data_to_insert['ozono'] = data_to_insert.ozono.apply(remove_Nan)

In [ ]:
import psycopg2 as pg
from psycopg2 import sql
cur = conn.cursor()
for index, row in data_to_insert.iterrows():
    insert_query = "insert into air_quality(date, pm25, pm10, dioxido_azufre, ozono, dioxido_nitrogeno,monoxido_carbono) VALUES ('{0}', '{1}', '{2}', {3}, {4}, {5}, {6})".format(row.fecha, 0, 0, 0,  row.ozono, row.dioxido, 0)
    cur.execute(insert_query)
conn.commit()
conn.close()